In [1]:
import pandas as pd
import glob
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
import tsfel
from joblib import Parallel, delayed
from sklearn.metrics import accuracy_score

In [2]:
import os
import pandas as pd
import numpy as np

# Base path to the directory containing the folders
base_path = "Combined/Train"  # Update this to the path where your folders are located

# Define the folders and corresponding labels
folders = {
    'WALKING': 1,
    'WALKING_UPSTAIRS': 2,
    'WALKING_DOWNSTAIRS': 3,
    'SITTING': 4,
    'STANDING': 5,
    'LAYING': 6
}

# Initialize lists to hold data and labels
data_frames = []
labels = []

# Loop through each folder and file to read the data
for folder, label in folders.items():
    folder_path = os.path.join(base_path, folder)  # Combine base path with the folder name
    # List all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):  # Ensure we only read CSV files
            # Read the CSV file
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)
            
            # Calculate the total acceleration magnitude
            df['total_acceleration'] = np.sqrt(df['accx']**2 + df['accy']**2 + df['accz']**2)
            
            # Append data to the data list
            data_frames.append(df)
            
            # Append corresponding label to the labels list
            labels.extend([label] * len(df))

# Concatenate all data frames into one
X_train = pd.concat(data_frames, ignore_index=True)

# Convert labels to a DataFrame
y_train = pd.DataFrame(labels, columns=['Activity'])

# Save combined data to CSV
X_train.to_csv('X_train_combined.csv', index=False)
y_train.to_csv('y_train_combined.csv', index=False)


In [3]:
import os
import pandas as pd
import numpy as np

# Base path to the directory containing the folders
base_path = "Combined/Test"  # Update this to the path where your folders are located

# Define the folders and corresponding labels
folders = {
    'WALKING': 1,
    'WALKING_UPSTAIRS': 2,
    'WALKING_DOWNSTAIRS': 3,
    'SITTING': 4,
    'STANDING': 5,
    'LAYING': 6
}

# Initialize lists to hold data and labels
data_frames = []
labels = []

# Loop through each folder and file to read the data
for folder, label in folders.items():
    folder_path = os.path.join(base_path, folder)  # Combine base path with the folder name
    # List all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):  # Ensure we only read CSV files
            # Read the CSV file
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)
            
            # Calculate the total acceleration magnitude
            df['total_acceleration'] = np.sqrt(df['accx']**2 + df['accy']**2 + df['accz']**2)
            
            # Append data to the data list
            data_frames.append(df)
            
            # Append corresponding label to the labels list
            labels.extend([label] * len(df))

# Concatenate all data frames into one
X_test = pd.concat(data_frames, ignore_index=True)

# Convert labels to a DataFrame
y_test = pd.DataFrame(labels, columns=['Activity'])

# Save combined data to CSV
X_test.to_csv('X_test_combined.csv', index=False)
y_test.to_csv('y_test_combined.csv', index=False)

In [40]:
X_test.shape

(188608, 4)

In [34]:
# Define the label mapping
activity_labels = {
    'walking': 1,
    'walkingupstair': 2,
    'walkingdownstair': 3,
    'sitting': 4,
    'standing': 5,
    'laying': 6
}

# List all CSV files in the directory
file_paths = glob.glob('data_scripts/our_data/*.csv')

# Initialize empty lists to store the data and labels
X_test_list = []
y_test_list = []

# Iterate through each file
for file_path in file_paths:
    # Extract the activity name from the file name
    file_name = file_path.split('/')[-1].lower()
    activity_name = file_name.split('_')[-1].replace('.csv', '')

    # Determine the label for the activity
    label = activity_labels.get(activity_name)
    
    if label is not None:
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Take only alternate rows and limit to 500 rows
        
        
        # Add the data and corresponding label to the lists
        X_test_list.append(df)
        y_test_list.extend([label] * len(df))

# Concatenate all dataframes to form X_test
X_test = pd.concat(X_test_list, ignore_index=True)

# Convert the labels list to a DataFrame to form y_test
y_test = pd.DataFrame(y_test_list, columns=['Activity'])

# Remove the 'time' column from X_train if it exists
if 'time' in X_test.columns:
    X_test = X_test.drop(columns=['time'])

# Example: Displaying the first few rows of X_test and y_test
print("First few rows of X_test:")
print(X_test.head())

print("\nFirst few rows of y_test:")
print(y_test.head())

First few rows of X_test:
   ax (m/s^2)  ay (m/s^2)  az (m/s^2)  aT (m/s^2)
0     -0.5037     -0.1326     -0.0406       0.522
1     -0.2844     -0.2044     -0.0087       0.350
2      0.1120     -0.3033      0.0481       0.327
3      0.3042     -0.2945     -0.0081       0.423
4      0.4825     -0.2595     -0.0304       0.549

First few rows of y_test:
   Activity
0         6
1         6
2         6
3         6
4         6


In [79]:
import pandas as pd
import glob

# Define the label mapping
activity_labels = {
    'walking': 1,
    'walkingupstair': 2,
    'walkingdownstair': 3,
    'sitting': 4,
    'standing': 5,
    'laying': 6
}

# List all CSV files in the directory
file_paths = glob.glob('data_scripts/our_data/*.csv')

# Initialize empty lists to store the data and labels
X_test_list = []
y_test_list = []

# Function to resample data to 50 Hz
def resample_to_50hz(df):
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'], errors='coerce')
        df = df.set_index('time').resample('20ms').mean().interpolate()
    return df

# Iterate through each file
for file_path in file_paths:
    # Extract the activity name from the file name
    
    file_name = file_path.split('/')[-1].lower()
    activity_name = file_name.split('_')[-1].replace('.csv', '')

    # Determine the label for the activity
    label = activity_labels.get(activity_name)
    
    if label is not None:
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Take only alternate rows but make sure to keep a minimum of rows
        if len(df) >= 500:
            df = df.iloc[::2, :].head(500)
        else:
            df = df.iloc[::2, :]  # keep all rows if less than 500
        
        # Add the resampled data and corresponding label to the lists
        # print("Appending: ",file_path)
        X_test_list.append(df)
        y_test_list.extend([label] * len(df))

# Concatenate all dataframes to form X_test
X_test = pd.concat(X_test_list, ignore_index=True)

# Convert the labels list to a DataFrame to form y_test
y_test = pd.DataFrame(y_test_list, columns=['Activity'])

# Remove the 'time' column from X_test if it exists
if 'time' in X_test.columns:
    X_test = X_test.drop(columns=['time'])

# # Example: Displaying the first few rows of X_test and y_test
# print("First few rows of X_test:")
# print(X_test.head())

# print("\nFirst few rows of y_test:")
# print(y_test.head())


In [76]:
len(X_test)

12000

In [77]:
y_test

,Activity
0,6
1,6
2,6
3,6
4,6
...,...
11995,2
11996,2
11997,2
11998,2


In [40]:
# file_paths

In [80]:
X_test

,ax (m/s^2),ay (m/s^2),az (m/s^2),aT (m/s^2)
0,-0.5037,-0.1326,-0.0406,0.522
1,0.1120,-0.3033,0.0481,0.327
2,0.4825,-0.2595,-0.0304,0.549
3,0.4833,-0.2023,0.2009,0.561
4,0.5082,-0.2226,0.5205,0.761
...,...,...,...,...
11995,-1.0919,-1.6690,-0.8239,2.158
11996,-0.5762,-1.4886,-0.9101,1.838
11997,-0.1972,-1.0017,-0.4554,1.118
11998,-0.4641,-0.0419,-0.1781,0.499


In [81]:
# Reset column names to default integer-based indices
X_test.columns = range(X_test.shape[1])
X_train.columns = range(X_train.shape[1])
y_test.columns = range(y_test.shape[1])
y_train.columns = range(y_train.shape[1])


In [82]:
y_test[0].unique()

array([6, 4, 5, 1, 3, 2])

In [83]:
# Load the training and test data
X_train = pd.read_csv('X_train_combined.csv')
y_train = pd.read_csv('y_train_combined.csv')


# Initialize the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)

# Train the model
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print(class_report)

Accuracy: 0.09
Precision: 0.10
Recall: 0.09
Confusion Matrix:
[[ 116  292  500   12    0 1080]
 [ 119  309  592    7    0  973]
 [  69  216  569    4    0 1142]
 [  90   88 1800    0    0   22]
 [ 149  129 1634    0    0   88]
 [  51   85 1806    0    0   58]]
              precision    recall  f1-score   support

           1       0.20      0.06      0.09      2000
           2       0.28      0.15      0.20      2000
           3       0.08      0.28      0.13      2000
           4       0.00      0.00      0.00      2000
           5       0.00      0.00      0.00      2000
           6       0.02      0.03      0.02      2000

    accuracy                           0.09     12000
   macro avg       0.10      0.09      0.07     12000
weighted avg       0.10      0.09      0.07     12000



c:\Users\SIDDHARTH\OneDrive\VS\ES335_assignment_1\ML-Assignment-Gamble.ai\.conda\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\SIDDHARTH\OneDrive\VS\ES335_assignment_1\ML-Assignment-Gamble.ai\.conda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\SIDDHARTH\OneDrive\VS\ES335_assignment_1\ML-Assignment-Gamble.ai\.conda\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\U